In [1]:
from keras.datasets import mnist
from keras.utils import *
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
def preprocess(data):
    
    (xtrain, ytrain), (xtest, ytest) = data.load_data()
    

     #Split Dataset into train and test set
    xtrain = xtrain.reshape(60000,784)
    xtest = xtest.reshape(10000,784)

     #Make the value floats in [0:1] instead of int in [0:255]
    xtrain = xtrain.astype('float32')
    xtest = xtest.astype('float32')
    xtrain /= 255
    xtest /= 255
    digits = 10
     #The output variable is an integer from 0 to 9.Convert a class vector (integers) to binary class matrix.
    ytrain = to_categorical(ytrain, digits)
    ytest = to_categorical(ytest, digits)
     
    return (xtrain, ytrain), (xtest, ytest)

def get_model(input_shape):
    neuralnet = Sequential()
    neuralnet.add(Dense(512, input_shape=input_shape, activation='relu'))
    
    neuralnet.add(Dense(512, activation='relu'))
    neuralnet.add(Dense(128, activation='relu'))
    neuralnet.add(Dense(100, activation='relu'))
    neuralnet.add(Dense(10, activation='softmax'))
    return neuralnet

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(xtrain, ytrain), (xtest, ytest) = preprocess(mnist)
model = get_model((784,))

In [3]:
print("#### Training ####\n")
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.fit(xtrain, ytrain, validation_data=(xtest, ytest), epochs=25, batch_size=250, verbose=2)

#### Training ####

Train on 60000 samples, validate on 10000 samples
Epoch 1/25
 - 19s - loss: 0.3214 - acc: 0.8995 - val_loss: 0.1301 - val_acc: 0.9609
Epoch 2/25
 - 17s - loss: 0.1049 - acc: 0.9679 - val_loss: 0.1104 - val_acc: 0.9669
Epoch 3/25
 - 17s - loss: 0.0683 - acc: 0.9792 - val_loss: 0.0793 - val_acc: 0.9749
Epoch 4/25
 - 17s - loss: 0.0488 - acc: 0.9849 - val_loss: 0.0773 - val_acc: 0.9770
Epoch 5/25
 - 17s - loss: 0.0353 - acc: 0.9890 - val_loss: 0.0720 - val_acc: 0.9819
Epoch 6/25
 - 18s - loss: 0.0285 - acc: 0.9914 - val_loss: 0.0983 - val_acc: 0.9763
Epoch 7/25
 - 19s - loss: 0.0234 - acc: 0.9929 - val_loss: 0.0812 - val_acc: 0.9787
Epoch 8/25
 - 11s - loss: 0.0198 - acc: 0.9940 - val_loss: 0.0945 - val_acc: 0.9789
Epoch 9/25
 - 10s - loss: 0.0165 - acc: 0.9950 - val_loss: 0.0861 - val_acc: 0.9800
Epoch 10/25
 - 10s - loss: 0.0161 - acc: 0.9954 - val_loss: 0.0938 - val_acc: 0.9794
Epoch 11/25
 - 10s - loss: 0.0135 - acc: 0.9957 - val_loss: 0.1384 - val_acc: 0.9762
Epoc

In [4]:
print("#### Let's Test on 1 sample ####\n")
print("\n#### Actual Vector ####\n" + str(ytest[0:10]))
y_test_pred = model.predict(xtest[0:10])
acc = model.evaluate(xtest, ytest, verbose=0)
print('Test Acc:', acc[1])

#### Let's Test on 1 sample ####


#### Actual Vector ####
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Test Acc: 0.9774


In [ ]:
import cv2
import numpy as np
import math
from IPython.display import Image
from matplotlib import pyplot as plt
from scipy import ndimage
y_test_pred=np.zeros((10,10))
k=0    
def form(img):
    def getBestShift(img):
        cy,cx = ndimage.measurements.center_of_mass(img)
        rows,cols = img.shape
        shiftx = np.round(cols/2.0-cx).astype(int)
        shifty = np.round(rows/2.0-cy).astype(int)
        return shiftx,shifty
    def shift(img,sx,sy):
        rows,cols = img.shape
        M = np.float32([[1,0,sx],[0,1,sy]])
        shifted = cv2.warpAffine(img,M,(cols,rows))
        return shifted
    gray=img
    gray = cv2.resize(255-gray, (28, 28))
    ret, gray = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)
    while np.sum(gray[0]) == 0:
            gray = gray[1:]
    while np.sum(gray[:,0]) == 0:
            gray = np.delete(gray,0,1)
    while np.sum(gray[-1]) == 0:
            gray = gray[:-1]
    while np.sum(gray[:,-1]) == 0:
            gray = np.delete(gray,-1,1)
    rows,cols = gray.shape
    if rows > cols:
            factor = 20.0/rows
            rows = 20
            cols = int(round(cols*factor))
            gray = cv2.resize(gray, (cols,rows))
    else:
            factor = 20.0/cols
            cols = 20
            rows = int(round(rows*factor))
            gray = cv2.resize(gray, (cols, rows))
    colsPadding = (int(math.ceil((28-cols)/2.0)),int(math.floor((28-cols)/2.0)))
    rowsPadding = (int(math.ceil((28-rows)/2.0)),int(math.floor((28-rows)/2.0)))
    gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant')
    shiftx,shifty = getBestShift(gray)
    shifted = shift(gray,shiftx,shifty)
    gray = shifted
    flatten = gray.flatten() / 255.0
    return gray
img = cv2.imread('test2.png')
img2gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(img2gray, 180, 255, cv2.THRESH_BINARY_INV)
_, contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
index=1
for contour in contours:
    [x, y, w, h] = cv2.boundingRect(contour)
    cv2.rectangle(img,(x,y),(x+w,y+h),(100,255,100),2)
    cropped = mask[y :y +  h , x : x + w]
    s =str(index) + '.png'
    if w < 30 and h < 30:
        continue
    '''
    constant= cv2.copyMakeBorder(cropped,10,10,10,10,cv2.BORDER_CONSTANT,(0,0,0))
    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.dilate(constant,kernel,iterations = 1)
    erosion = cv2.erode(erosion,kernel,iterations = 1)
    erosion = cv2.dilate(erosion,kernel,iterations = 1)
    resized = cv2.resize(erosion, dim, interpolation = cv2.INTER_AREA)
    '''
    resized=form(cropped)
   

    img2 = resized
    img1=img2.reshape(1,784)
    y_test_pred[k] = model.predict(img1)
    k=k+1
    cv2.imwrite(s ,resized)
    imgd=Image(s)
    display(imgd)
   
    index = index + 1
x=np.round(y_test_pred)
m=0
final=np.zeros(10)
for i in range(10):
              for j in range(10):
                    if(x[i,j]==1):
                        final[m]=j
                        m=m+1
print(final)
cv2.imshow('captcha_result', img)
cv2.waitKey()

[0. 9. 1. 2. 3. 7. 4. 8. 6. 5.]
